In [3]:
import concurrent.futures #for managing concurrent tasks
import queue #creating a queue for the data
import requests #for making web requests
from bs4 import BeautifulSoup #using the BeautifulSoup library to extract hyperlinks (links) from the HTML

# Constants
Producers = 2
Consumers = 4
max_queue = 50

# Shared queue for storing HTML content
html_queue = queue.Queue(maxsize=max_queue)

# Function to fetch HTML content from a URL
def fetch(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

# Function to parse HTML content and extract hyperlinks
def parse_html(html, url):
    try:
        soup = BeautifulSoup(html, 'html.parser')
        links = [a.get('href') for a in soup.find_all('a')]
        return links
    except Exception as e:
        print(f"Error parsing {url}: {e}")
        return []

# Producer function
def producer(url_list):
    for url in url_list:
        html = fetch(url)
        if html:
            html_queue.put((html, url))

# Consumer function
def consumer():
    while True:
        try:
            html, url = html_queue.get(timeout=5)  # Adjust the timeout as needed
            links = parse_html(html, url)
            print(f"Links found in {url}: {links}")
        except queue.Empty:
            break

if __name__ == "__main__":
    urls = []  # Replace with your list of URLs

    with concurrent.futures.ThreadPoolExecutor(max_workers=Producers) as producer_executor:
        producer_executor.map(producer, [urls])

    with concurrent.futures.ThreadPoolExecutor(max_workers=Consumers) as consumer_executor:
        for _ in range(Consumers):
            consumer_executor.submit(consumer)

